# Direct Radio Access to Test Byte Files

In [1]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime

In [2]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

('tagnet: ', '/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '/mnt/Open/TagNet/notebooks')
tagnet driver version 0.0.25


In [3]:
sys.path.insert(0,"../tagfuse")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_config, radio_show_config

('si446x version: ', '0.2.6')


In [4]:
from tagfuse.tagfuseargs import __version__
'tagfuse', __version__

('tagfuse', '0.0.33')

In [5]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

Test Start Time: 2018-03-26 22:27:03.776854


##  Start up Radio

In [6]:
radio = radio_start()

/mnt/Open/TagNet/tagfuse/../si446x/si446x/si446xradio.py:711: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(GPIO_SDN,GPIO.OUT)   #  [sdn]


In [7]:
radio_show_config(radio.dump_radio())


int_ctl_group_s: 4
073b 2328 
 enable:  CHIP_INT_STATUS_EN MODEM_INT_STATUS_EN PH_INT_STATUS_EN ph_pend:  PACKET_SENT PACKET_RX CRC_ERROR TX_FIFO_ALMOST_EMPTY RX_FIFO_ALMOST_FULL modem_pend:  INVALID_SYNC PREAMBLE_DETECT SYNC_DETECT chip_pend:  FIFO_UNDERFLOW_OVERFLOW_ERROR CMD_ERROR

global_group_s: 10
5200 1860 0000 0160 0000 
Container:
    xo_tune = 82
    clk_cfg = Container:
        divided_clk_en = 'DISABLE'
        divided_clk_sel = 'DIV_1'
        clk_32k_sel = 'OFF'
    low_batt_thresh = 24
    global_config = Container:
        sequencer_mode = 'FAST'
        fifo_mode = 'SPLIT_FIFO'
        protocol = 'GENERIC'
        power_mode = 'HIGH_PERF'
    wut_config = Container:
        wut_ldc_enable = 'DISABLE'
        wut_cal_period = '_1_SEC'
        wut_lbd_en = 'DISABLE'
        wut_en = 'DISABLE'
        cal_en = 'DISABLE'
    wut_m = 1
    wut_r = Container:
        RESERVED_WRITE_ONE = 'ONE'
        wut_sleep = 'SLEEP'
        wut_r_ = 0
    wut_ldc = 0
    wut_cal = 0

s

### Set Defaults

In [8]:
# default paramters
MAX_WAIT            = 1
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 50
SHORT_DELAY         = 1

### Read Test Bytes 

In [9]:
STOP


NameError: name 'STOP' is not defined

In [ ]:
TestByteName = 'echo'

In [ ]:
test_byte_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv(TestByteName),
                            TagTlv('byte'),
                            TagTlv(tlv_types.OFFSET, 1),
                            TagTlv(tlv_types.SIZE, 100),
                             ])

In [ ]:
req_obj = TagGet(test_byte_name)
#    print(get_gps_xyz.name)
req_msg = req_obj.build()

In [ ]:
len(req_msg), req_obj.header, req_obj.name, req_obj.payload

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
len(rsp_msg), rssi

In [ ]:
if rsp_msg:
    #        print(hexlify(rsp_msg))
    rsp_obj = TagMessage(rsp_msg)
    #        print(rsp_obj.header.options.param.error_code)
    #        print(rsp_obj.payload)
    if rsp_obj.payload:
        print(len(rsp_msg), rsp_obj.header, rsp_obj.name, rsp_obj.payload)

In [ ]:
rsp_obj.payload[1].value()

In [ ]:
sstatus, rstatus

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
rsp_msg, rssi

In [ ]:
for x in range(10):
    print(datetime.datetime.now())
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    #print(sstatus)
    print(datetime.datetime.now())
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    print(rssi, len(rsp_msg))
    print(rstatus)

In [ ]:
radio.spi.trace.display(radio.spi.trace.filter(count=-10))

### Write Test Bytes 

In [ ]:
STOP


In [ ]:
TestByteName = 'echo'
TestByteBuf  = bytearray('0123456789' * 10)
#TestByteBuf  = bytearray('e' * 22)

In [ ]:
len(TestByteBuf), TestByteBuf

In [ ]:
test_byte_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                          TagTlv('tag'),
                          TagTlv('.test'),
                          TagTlv(TestByteName),
                          TagTlv('byte'),
                          TagTlv(tlv_types.OFFSET, 1),
                          TagTlv(tlv_types.SIZE, 10),
                             ])

In [ ]:
req_obj = TagPut(test_byte_name, pl=TestByteBuf)
#    print(get_gps_xyz.name)
req_msg = req_obj.build()

In [ ]:
 len(req_msg), req_obj.name

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
len(rsp_msg), rssi

In [ ]:
if rsp_msg:
    #        print(hexlify(rsp_msg))
    rsp_obj = TagMessage(rsp_msg)
    #        print(rsp_obj.header.options.param.error_code)
    #        print(rsp_obj.payload)
    if rsp_obj.payload:
        print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

In [ ]:
len(rsp_msg), rsp_msg, rsp_obj.payload

In [ ]:
rsp_obj.header, len(rsp_obj.name), rsp_obj.name, rsp_obj.payload

In [ ]:
'send', sstatus, 'receive', rstatus

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
sstatus

In [ ]:
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
rsp_msg, rssi

In [ ]:
radio.trace.display(radio.trace.filter(count=-10))

In [ ]:
a,b,c = sstatus[6:9]

In [ ]:
a.items()[0][1]

## Get Tag's RSSI value

In [ ]:
STOP

In [ ]:
rssi_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('rssi'),
                             ])

In [ ]:
get_rssi = TagGet(rssi_name)
#    print(get_gps_xyz.name)
req_msg = get_rssi.build()

In [ ]:
get_rssi.name, len(req_msg)

In [ ]:
for i in range(10):
    rstatus = radio_send_msg(radio, req_msg, RADIO_POWER);
    rsp_msg, rssi, sstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if rssi == -1:
        break
    print(rssi, len(rsp_msg))

In [ ]:
rssi, rsp_msg

In [ ]:
if rsp_msg:
    #        print(hexlify(rsp_msg))
    rsp_obj = TagMessage(rsp_msg)
    #        print(rsp_obj.header.options.param.error_code)
    #        print(rsp_obj.payload)
    if rsp_obj.payload:
        print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

In [ ]:
status = radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
sstatus

In [ ]:
radio.spi.trace.display(radio.spi.trace.filter(count=-100))

In [ ]:
rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 30)

In [ ]:
if rsp_msg:
    #        print(hexlify(rsp_msg))
    rsp_obj = TagMessage(rsp_msg)
    #        print(rsp_obj.header.options.param.error_code)
    #        print(rsp_obj.payload)
    if rsp_obj.payload:
        print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

### Write Zeros

In [ ]:
STOP

In [ ]:
#<node_id>   "tag"  "test"   "zero"   "byte"
def read_test_bytes(testfile, pos, num):
    zero_byte_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv(testfile),
                            TagTlv('byte'),
                            TagTlv(tlv_types.OFFSET, pos),
                            TagTlv(tlv_types.SIZE, num),
                             ])
    get_zero_byte = TagGet(zero_byte_name)
#    print(get_gps_xyz.name)
    req_msg = get_zero_byte.build()
    radio_send_msg(radio, req_msg, RADIO_POWER);
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if rsp_msg:
#        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
#        print(rsp_obj.header.options.param.error_code)
#        print(rsp_obj.payload)
        if rsp_obj.payload:
            error, offset, amt, block = payload2values(rsp_obj.payload,
                                  [tlv_types.ERROR,
                                   tlv_types.OFFSET,
                                   tlv_types.SIZE,
                                   tlv_types.BLOCK,
                                  ])
            seta = set(block)
            # print(seta)
            if (len(seta) > 1):
                print('check error', seta, amt, hexlify(block))
                amt = 0
            return error, offset, amt, block
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))
#    else:
#        print('TIMEOUT')
    return None

In [ ]:
start = datetime.datetime.now()
for x in range(1,1000):
    for y in range(1,10):
        sleep(SHORT_DELAY)
        return_values = read_test_bytes('zero', 1, 1)
        if return_values:
            error, offset, amt, block = return_values
            # print('error: {}'.format(error))
            if error and error is not tlv_errors.SUCCESS:
                print("\r{} {}".format(datetime.now() - start, error), end="")
                continue
            print('got bytes, len: {}, count: {}, offset: {}'.format(len(block),
                                                                    amt,
                                                                    offset))
        else:
            print('timeout')
            
print("\ntime:{}".format(datetime.now() - start))

## Listen for packets

In [ ]:
for i in range(200000):
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 3000)
    if rsp_msg:
        print(datetime.datetime.now(), rssi, len(rsp_msg), hexlify(rsp_msg))
        #rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        #if rsp_obj.payload:
        #    print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

2018-03-26 22:27:53.646129 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:27:53.739518 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:27:53.861361 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:27:53.954337 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:27:54.076343 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:27:54.161293 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:27:54.288743 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:27:54.373805 219 29 1c1094190506ffffffffffff010

2018-03-26 22:28:00.635085 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:00.724044 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:00.845325 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:00.939806 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:01.058429 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:01.148225 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:01.267919 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:01.357684 219 29 1c1094190506ffffffffffff010

2018-03-26 22:28:07.477465 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:07.593878 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:07.684568 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:07.804791 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:07.901588 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:08.015051 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:08.104463 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:08.226509 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04b

2018-03-26 22:28:14.380599 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:14.474737 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:14.595775 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:14.689876 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:14.811086 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:14.905741 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:15.027085 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:15.122095 219 29 1c1094190506ffffffffffff010

2018-03-26 22:28:21.308567 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:21.402266 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:21.527343 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:21.632848 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:21.754549 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:21.848283 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:28:21.967440 223 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:28:22.067120 219 29 1c1094190506ffffffffffff010

...

2018-03-26 22:44:19.109206 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:19.224427 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:19.327125 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:19.447023 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:19.549773 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:19.665451 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:19.768140 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:19.882101 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04b

2018-03-26 22:44:26.042858 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:26.132893 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:26.254875 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:26.344984 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:26.465256 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:26.555995 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:26.677706 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:26.767904 219 29 1c1094190506ffffffffffff010

2018-03-26 22:44:32.950824 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:33.066029 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:33.164344 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:33.278903 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:33.376363 218 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:33.491165 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:33.594606 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:33.708813 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04b

2018-03-26 22:44:39.744849 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:39.862807 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:39.958958 218 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:40.079089 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:40.170789 218 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:40.290136 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:40.381053 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:40.495106 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04b

2018-03-26 22:44:46.645277 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:46.733637 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:46.859841 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:46.949161 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:47.069184 221 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:47.159757 219 29 1c1094190506ffffffffffff0103746167010273640201000103696d67
2018-03-26 22:44:47.280171 222 65 409180190506ffffffffffff0103746167010273640201000103696d670a04820102000101760a04ba0102000101760a04bb0102000101610a04d0010200010178
2018-03-26 22:44:47.373677 219 29 1c1094190506ffffffffffff010

## Get Chip Status

In [ ]:
STOP

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
STOP

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()